In [1]:
import sys; sys.path.insert(0, '../../')

In [2]:
from pathlib import Path
from models.snn.factory import build_snn
from models.snn.train import train_model, evaluate_model
from models.snn.tuning import tune_hyperparameters
from models.preprocessing import load_preprocessed

2026-01-10 00:40:54.012841: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2026-01-10 00:40:54.012888: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2026-01-10 00:40:57.014446: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2026-01-10 00:40:58.496470: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2026-01-10 00:40:58.496607: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: l

In [3]:
splits, class_names = load_preprocessed('../../data/processed/')
X_train, y_train = splits['train']
X_val, y_val = splits['val']
X_test, y_test = splits['test']

Loaded train: X=(818, 614400), y=(818, 3)
Loaded val: X=(235, 614400), y=(235, 3)
Loaded test: X=(118, 614400), y=(118, 3)
Loaded class names: ['mark', 'marvin', 'yannes']


In [4]:
checkpoint_dir = Path('../../checkpoints/snn')
env_path = str((checkpoint_dir / "best_hparams.env").absolute())

In [5]:
best_config, analysis = tune_hyperparameters(
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    n_features=X_train.shape[1],
    n_classes=len(class_names),
    max_epochs=5,
    num_samples=20,
    project_name="snn_ray_tune",
 )

[Ray Tune] Initializing Ray runtime...


ValueError: After taking into account object store and redis memory usage, the amount of memory on this node available for tasks and actors (0.09 GB) is less than 0% of total. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).

In [ ]:
print("\nBest hyperparameters found by Ray Tune:\n")
print(f"  Hidden neurons: {best_config['n_neurons_hidden']}")
print(f"  Synapse fast: {best_config.get('synapse_fast')}")
print(f"  Synapse slow: {best_config.get('synapse_slow')}")
print(f"  Learning rate: {best_config['learning_rate']:.2e}")
print(f"  Batch size: {best_config['batch_size']}")

best_hidden = int(best_config['n_neurons_hidden'])
best_syn_fast = float(best_config.get('synapse_fast'))
best_syn_slow = float(best_config.get('synapse_slow'))
best_lr = float(best_config['learning_rate'])
best_batch_sz = int(best_config['batch_size'])

In [ ]:
print(f"\nSaving best hyperparameters to {env_path} ...")
with open(env_path, "w") as f:
    f.write(f"N_NEURONS_HIDDEN={best_hidden}\n")
    f.write(f"SYNAPSE_FAST={best_syn_fast}\n")
    f.write(f"SYNAPSE_SLOW={best_syn_slow}\n")
    f.write(f"LEARNING_RATE={best_lr}\n")
    f.write(f"BATCH_SIZE={best_batch_sz}\n")
    f.write(f"N_FEATURES={X_train.shape[1]}\n")
    f.write(f"N_CLASSES={len(class_names)}\n")
print("Best hyperparameters saved.\n")